# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 6 2022 3:25PM,248108,10,8619255-SD,Eywa Pharma Inc.,Released
1,Oct 6 2022 3:18PM,248107,10,SOTAH0000804,"Nextsource Biotechnology, LLC",Released
2,Oct 6 2022 3:10PM,248106,10,SNAP32220099,"Snap Medical Industries, LLC",Released
3,Oct 6 2022 3:10PM,248106,10,SNAP32228756,"Snap Medical Industries, LLC",Released
4,Oct 6 2022 3:10PM,248106,10,SNAP32246939,"Snap Medical Industries, LLC",Released
5,Oct 6 2022 3:03PM,248079,10,MSP214514,"Methapharm, Inc.",Released
6,Oct 6 2022 3:03PM,248079,10,MSP214682,"Methapharm, Inc.",Released
7,Oct 6 2022 2:48PM,248098,16,8618662,Sartorius Bioprocess Solutions,Released
8,Oct 6 2022 2:46PM,248097,16,8618665,Sartorius Bioprocess Solutions,Released
9,Oct 6 2022 2:43PM,248095,16,8614994,Sartorius Lab Products and Service,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,248097,Released,1
23,248098,Released,1
24,248106,Released,3
25,248107,Released,1
26,248108,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
248097,NaN,NaN,1.0
248098,NaN,NaN,1.0
248106,NaN,NaN,3.0
248107,NaN,NaN,1.0
248108,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248000,6.0,4.0,10.0
248031,0.0,2.0,10.0
248037,3.0,4.0,1.0
248046,0.0,0.0,1.0
248049,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248000,6,4,10
248031,0,2,10
248037,3,4,1
248046,0,0,1
248049,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248000,6,4,10
1,248031,0,2,10
2,248037,3,4,1
3,248046,0,0,1
4,248049,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248000,6,4,10
1,248031,,2,10
2,248037,3,4,1
3,248046,,,1
4,248049,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 6 2022 3:25PM,248108,10,Eywa Pharma Inc.
1,Oct 6 2022 3:18PM,248107,10,"Nextsource Biotechnology, LLC"
2,Oct 6 2022 3:10PM,248106,10,"Snap Medical Industries, LLC"
5,Oct 6 2022 3:03PM,248079,10,"Methapharm, Inc."
7,Oct 6 2022 2:48PM,248098,16,Sartorius Bioprocess Solutions
8,Oct 6 2022 2:46PM,248097,16,Sartorius Bioprocess Solutions
9,Oct 6 2022 2:43PM,248095,16,Sartorius Lab Products and Service
11,Oct 6 2022 2:26PM,248085,20,"ACI Healthcare USA, Inc."
12,Oct 6 2022 2:25PM,248090,10,Yusen – 3D Matrix
13,Oct 6 2022 2:16PM,248082,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 6 2022 3:25PM,248108,10,Eywa Pharma Inc.,,,1
1,Oct 6 2022 3:18PM,248107,10,"Nextsource Biotechnology, LLC",,,1
2,Oct 6 2022 3:10PM,248106,10,"Snap Medical Industries, LLC",,,3
3,Oct 6 2022 3:03PM,248079,10,"Methapharm, Inc.",,,2
4,Oct 6 2022 2:48PM,248098,16,Sartorius Bioprocess Solutions,,,1
5,Oct 6 2022 2:46PM,248097,16,Sartorius Bioprocess Solutions,,,1
6,Oct 6 2022 2:43PM,248095,16,Sartorius Lab Products and Service,,2,
7,Oct 6 2022 2:26PM,248085,20,"ACI Healthcare USA, Inc.",,,1
8,Oct 6 2022 2:25PM,248090,10,Yusen – 3D Matrix,,,1
9,Oct 6 2022 2:16PM,248082,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 3:25PM,248108,10,Eywa Pharma Inc.,1,,
1,Oct 6 2022 3:18PM,248107,10,"Nextsource Biotechnology, LLC",1,,
2,Oct 6 2022 3:10PM,248106,10,"Snap Medical Industries, LLC",3,,
3,Oct 6 2022 3:03PM,248079,10,"Methapharm, Inc.",2,,
4,Oct 6 2022 2:48PM,248098,16,Sartorius Bioprocess Solutions,1,,
5,Oct 6 2022 2:46PM,248097,16,Sartorius Bioprocess Solutions,1,,
6,Oct 6 2022 2:43PM,248095,16,Sartorius Lab Products and Service,,2,
7,Oct 6 2022 2:26PM,248085,20,"ACI Healthcare USA, Inc.",1,,
8,Oct 6 2022 2:25PM,248090,10,Yusen – 3D Matrix,1,,
9,Oct 6 2022 2:16PM,248082,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 3:25PM,248108,10,Eywa Pharma Inc.,1,,
1,Oct 6 2022 3:18PM,248107,10,"Nextsource Biotechnology, LLC",1,,
2,Oct 6 2022 3:10PM,248106,10,"Snap Medical Industries, LLC",3,,
3,Oct 6 2022 3:03PM,248079,10,"Methapharm, Inc.",2,,
4,Oct 6 2022 2:48PM,248098,16,Sartorius Bioprocess Solutions,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 3:25PM,248108,10,Eywa Pharma Inc.,1.0,NaN,NaN
1,Oct 6 2022 3:18PM,248107,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
2,Oct 6 2022 3:10PM,248106,10,"Snap Medical Industries, LLC",3.0,NaN,NaN
3,Oct 6 2022 3:03PM,248079,10,"Methapharm, Inc.",2.0,NaN,NaN
4,Oct 6 2022 2:48PM,248098,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 6 2022 3:25PM,248108,10,Eywa Pharma Inc.,1.0,0.0,0.0
1,Oct 6 2022 3:18PM,248107,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
2,Oct 6 2022 3:10PM,248106,10,"Snap Medical Industries, LLC",3.0,0.0,0.0
3,Oct 6 2022 3:03PM,248079,10,"Methapharm, Inc.",2.0,0.0,0.0
4,Oct 6 2022 2:48PM,248098,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1984634,21.0,10.0,0.0
15,496037,11.0,8.0,9.0
16,992351,3.0,2.0,0.0
19,992226,4.0,0.0,0.0
20,496156,2.0,0.0,0.0
21,248082,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1984634,21.0,10.0,0.0
1,15,496037,11.0,8.0,9.0
2,16,992351,3.0,2.0,0.0
3,19,992226,4.0,0.0,0.0
4,20,496156,2.0,0.0,0.0
5,21,248082,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,21.0,10.0,0.0
1,15,11.0,8.0,9.0
2,16,3.0,2.0,0.0
3,19,4.0,0.0,0.0
4,20,2.0,0.0,0.0
5,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,21.0
1,15,Released,11.0
2,16,Released,3.0
3,19,Released,4.0
4,20,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,0.0,9.0,0.0,0.0,0.0,0.0
Executing,10.0,8.0,2.0,0.0,0.0,0.0
Released,21.0,11.0,3.0,4.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,0.0,9.0,0.0,0.0,0.0,0.0
1,Executing,10.0,8.0,2.0,0.0,0.0,0.0
2,Released,21.0,11.0,3.0,4.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,0.0,9.0,0.0,0.0,0.0,0.0
1,Executing,10.0,8.0,2.0,0.0,0.0,0.0
2,Released,21.0,11.0,3.0,4.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()